# Bibliotek

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from skyfield.api import EarthSatellite, load
from datetime import datetime, timedelta
from PIL import Image
import plotly.graph_objects as go

# Funksjoner og Konstanter

In [ ]:
earth_radius = 6371.137 #km

def calc_sun_sync_params(altitudes_km):
    """
    Calculate sun-synchronous orbital parameters for satellites at given altitudes.

    Parameters:
    altitudes_km (list of float): List of altitudes in kilometers.

    Returns:
    pd.DataFrame: A DataFrame containing the orbital parameters for each altitude.
    """
    R_EARTH = 6378.137e3  # Earth's equatorial radius in meters
    mu = 3.986004418e14  # Earth's gravitational parameter in m^3 s^-2
    J2 = 1.08263e-3  # Earth's J2 zonal harmonic for oblateness effect
    omega_earth = 2 * np.pi / (365.25 * 24 * 3600)  # Earth's angular velocity (rad/s)
    
    data = {
        "Altitude (km)": [],
        "Period (minutes)": [],
        "Revs per day": [],
        "Inclination (degrees)": [],
        "Semi-major Axis (meters)": [],
        "Eccentricity": [],
        "RAAN (degrees)": [],
        "Argument of Perigee (degrees)": []
    }
    
    for altitude_km in altitudes_km:
        if not isinstance(altitude_km, (int, float)):
            raise ValueError("All altitudes must be numeric values.")
        
        r = R_EARTH + altitude_km * 1e3  # Semi-major axis in meters
        T = 2 * np.pi * np.sqrt(r**3 / mu)  # Orbital period in seconds
        T_minutes = T / 60  # Convert period to minutes
        T_hours = T / 3600  # Convert period to hours
        revs_per_day = 24 / T_hours  # Calculate revolutions per day

        # Calculate inclination for a sun-synchronous orbit
        inclination = np.degrees(np.arccos(-2 * r**(7/2) * omega_earth / (3 * J2 * R_EARTH**2 * np.sqrt(mu))))

        # Set eccentricity to zero for a near-circular orbit
        eccentricity = 0.0

        # RAAN and Argument of Perigee are typically set to specific values for sun-synchronous orbits
        RAAN = 0.0  # Placeholder value
        argument_of_perigee = 0.0  # Placeholder value

        data["Altitude (km)"].append(altitude_km)
        data["Period (minutes)"].append(T_minutes)
        data["Revs per day"].append(revs_per_day)
        data["Inclination (degrees)"].append(inclination)
        data["Semi-major Axis (meters)"].append(r)
        data["Eccentricity"].append(eccentricity)
        data["RAAN (degrees)"].append(RAAN)
        data["Argument of Perigee (degrees)"].append(argument_of_perigee)
    
    df_orbital_params = pd.DataFrame(data)
    return df_orbital_params

def generate_tle(df):
    """
    Generate Two Line Elements (TLE) from orbital parameters in a DataFrame.

    Parameters:
    df (pd.DataFrame): DataFrame containing the orbital parameters.

    Returns:
    list of tuples: List of TLE lines and their descriptions.
    """
    tle_lines = []
    for index, row in df.iterrows():
        satellite_number = 99990 + index + 1  # Example satellite number
        epoch = "24304.50000000"  # Example epoch (YYDDD.DDDDDDDD)
        inclination = f"{row['Inclination (degrees)']:.4f}".rjust(8)
        raan = f"{row['RAAN (degrees)']:.4f}".rjust(8)
        eccentricity = f"{int(row['Eccentricity'] * 1e7):07d}"
        argument_of_perigee = f"{row['Argument of Perigee (degrees)']:.4f}".rjust(8)
        mean_anomaly = "000.0000"  # Placeholder value
        mean_motion = f"{row['Revs per day']:.8f}".rjust(11)
        revolution_number = f"{index + 1:05d}"

        line1 = f"1 {satellite_number}U 21001A   {epoch}  .00000000  00000-0  00000-0 0  9991"
        line2 = f"2 {satellite_number} {inclination} {raan} {eccentricity} {argument_of_perigee} {mean_anomaly} {mean_motion} {revolution_number}"

        label = f"Satellite {index + 1} ({row['Altitude (km)']} km)"
        tle_lines.append(((line1, line2), label))

    return tle_lines


def sphere(size, texture): 
    N_lat = int(texture.shape[0])
    N_lon = int(texture.shape[1])
    theta = np.linspace(0,2*np.pi,N_lat)
    phi = np.linspace(0,np.pi,N_lon)
    
    # Set up coordinates for points on the sphere
    x0 = size * np.outer(np.cos(theta),np.sin(phi))
    y0 = size * np.outer(np.sin(theta),np.sin(phi))
    z0 = size * np.outer(np.ones(N_lat),np.cos(phi))
    
    return x0, y0, z0

def create_satellite(tle_line1, tle_line2):
    return EarthSatellite(tle_line1, tle_line2)

def propagate_orbit(satellite, start_time, end_time, time_step):
    ts = load.timescale()
    times = ts.utc(start_time.year, start_time.month, start_time.day, 
                   start_time.hour, start_time.minute, start_time.second + np.arange(0, int((end_time - start_time).total_seconds()), time_step))
    geocentric = satellite.at(times)
    positions = geocentric.position.km
    velocities = geocentric.velocity.km_per_s
    return positions, velocities

def plot_orbits(tle_lines, time_step=60, time_delta=24):
    
    """
    Plots the orbits of satellites based on their TLE data.

    Parameters:
    tle_lines (list of tuples): A list of tuples where each tuple contains two strings representing the TLE lines of a satellite and a label for the satellite.
    time_step (int): The time step in seconds for propagating the satellite orbits. Default is 60 seconds.
    time_delta (int): The total time duration in hours for which the orbits are propagated. Default is 24 hours.

    Returns:
    None: The function displays an interactive 3D plot of the Earth and the satellite orbits using Plotly.
    """
    
    start_time = datetime.now()
    end_time = start_time + timedelta(hours=time_delta)
    positions_list = []
    velocities_list = []
    labels = []
    for (tle_line1, tle_line2), label in tle_lines:
        satellite = create_satellite(tle_line1, tle_line2)
        positions, velocities = propagate_orbit(satellite, start_time, end_time, time_step)
        positions_list.append(positions)
        velocities_list.append(velocities)
        labels.append(label)

    fig = go.Figure()
    # Plot the Earth
    texture = np.asarray(Image.open('temp.jpg')).T
    colorscale =[[0.0, 'rgb(30, 59, 117)'],
                 [0.1, 'rgb(46, 68, 21)'],
                 [0.2, 'rgb(74, 96, 28)'],
                 [0.3, 'rgb(115,141,90)'],
                 [0.4, 'rgb(122, 126, 75)'],
                 [0.6, 'rgb(122, 126, 75)'],
                 [0.7, 'rgb(141,115,96)'],
                 [0.8, 'rgb(223, 197, 170)'],
                 [0.9, 'rgb(237,214,183)'],
                 [1.0, 'rgb(255, 255, 255)']]              
    xs, ys, zs = sphere(earth_radius, texture)
    fig.add_trace(go.Surface(x=xs, y=ys, z=zs,
                      surfacecolor=texture,
                      colorscale=colorscale,
                      showscale=False,
                      hoverinfo='none',
                      name='Earth',
                      showlegend=True)
    )
    
    # Plot each satellite's orbit with different colors from the colormap
    cmap = plt.get_cmap('cool')  # Use the 'cool' colormap
    for i, (positions, velocities, label) in enumerate(zip(positions_list, velocities_list, labels)):
        color = cmap(i / len(positions_list))  # Get a color from the colormap
        hover_text = [f"Velocity: {np.linalg.norm(vel):.2f} km/s<br>Distance: {np.linalg.norm(pos)-earth_radius:.2f} km" for vel, pos in zip(velocities.T, positions.T)]
        fig.add_trace(go.Scatter3d(
            x=positions[0], y=positions[1], z=positions[2],
            mode='lines',
            name=label,
            line=dict(color='rgb({}, {}, {})'.format(*[int(c * 255) for c in color[:3]])),
            text=hover_text,
            hoverinfo='text'
        ))

    # Set the layout for the plot
    fig.update_layout(
        title=dict(
            text='Satellite Orbits',
            font=dict(size=24, color='white'),
            xanchor='center', yanchor='top', x=0.5, y=0.95
        ),
        width=800,
        height=600,
        scene=dict(
            xaxis=dict(backgroundcolor='black', color='white', gridcolor='gray', title='x (km)'),
            yaxis=dict(backgroundcolor='black', color='white', gridcolor='gray', title='y (km)'),
            zaxis=dict(backgroundcolor='black', color='white', gridcolor='gray', title='z (km)'),
            bgcolor='black'
        ),
        legend=dict(
            x=0, y=1,  # Position the legend at the bottom right
            xanchor='left', yanchor='top'
        ),
        margin=dict(r=0, l=0, b=0, t=0)
    )

    # Show the plot
    fig.show()

# Analyse

In [ ]:
tle_lines = [
    # ISS (LEO)
    (('1 25544U 98067A   20334.54791667  .00016717  00000-0  10270-3 0  9000', 
      '2 25544  51.6442  21.4614 0001448  45.3583 314.7000 15.49112345  9000'), 'ISS (LEO)'),

    # GPS BIIR-2 (MEO)
    (('1 24876U 97035A   20334.54791667  .00000023  00000-0  00000-0 0  9993',
      '2 24876  54.8990  56.0000 0000001  56.0000  56.0000  2.00562500  9993'), 'GPS BIIR-2 (MEO)'),

    # Molniya 1-93 (HEO)
    (('1 24779U 97017A   20334.54791667  .00000023  00000-0  00000-0 0  9993',
      '2 24779  62.8000  56.0000 0000001  56.0000  56.0000  2.00562500  9993'), 'Molniya 1-93 (HEO)'),

    # SES-1 (Geostationary)
    (('1 36516U 10018A   20334.54791667  .00000023  00000-0  00000-0 0  9993',
      '2 36516   0.0180  92.0000 0000001  92.0000 268.0000  1.00270000  9993'), 'SES-1 (GEO)')
]

# Altitudes for which we want to calculate orbital parameters
altitudes = [400, 500, 600]
orbital_params = calc_sun_sync_params(altitudes)

print(orbital_params)

# Plot the orbits of the satellites
plot_orbits(generate_tle(orbital_params)) 